In [24]:
# This script is for analysing if feature and target are paralogs. The code can be integrated into figures.py
# Author: Yiyun

import sys
sys.path.append('../')

from src.ceres_infer.analyses import *
from ast import literal_eval

In [25]:
### Load data and setting
# set color
src_colors = {'CERES':(214/255, 39/255, 40/255, 1.0), #red
              'RNA-seq':(31/255, 119/255, 180/255, 1.0), #blue
              'CN':(255/255, 127/255, 14/255, 1.0), #orange
              'Mut':(44/255, 160/255, 44/255, 1.0), #green
              'Lineage':(188/255, 189/255, 34/255, 1.0), #yellow
              'nan':(220/255, 220/255, 220/255, 1.0)} #grey
# output directory
dir_out = '../out'

# load summary data
dir_in_res = '../out/20.0216 feat/reg_rf_boruta'
dir_in_anlyz = os.path.join(dir_in_res, 'anlyz_filtered')
df_featSummary = pd.read_csv(os.path.join(dir_in_anlyz, 'feat_summary.csv')) #feature summary
df_featSummary['feat_sources'] = df_featSummary['feat_sources'].apply(literal_eval)
df_featSummary['feat_genes'] = df_featSummary['feat_genes'].apply(literal_eval)

In [26]:
### Modified code
# parse geneset data and read line by line
def parseGeneset_m(fname):
    genesets = dict()
    f = open(fname)
    for x in f:
        gs_name = re.sub('\\t\\t.*\\n','',x)
        genes = re.sub('.*\\t\\t','',x).replace('\t\n','').split(sep='\t')
        genes = np.hstack(genes)
        genesets[gs_name] = genes
    f.close()
    
    return genesets

# not generating feat suammry; add paralog file; delete combined geneset
def anlyz_varExp_feats_m(feat_summary, gs_dir = '../out/', outdir_sub='../out'):
    # analyze features
    if(np.logical_not(os.path.exists(outdir_sub))): os.mkdir(outdir_sub)
    
    # paralog analysis
    genesets = parseGeneset_m('%s/paralog.txt' % gs_dir)
    sameGs_counts,sameGs_src_counts,feat_summary_annot = getGrpCounts(isInSameGS, isInSameGS_sources, feat_summary, genesets)
    plotGrpCounts(sameGs_counts, sameGs_src_counts, feat_summary_annot, 'in paralog list', outdir_sub)

In [27]:
### Just copied the code because importing wasn't successful
def gen_feat_pies(sameGrp_counts, sameGrp_src_counts, feat_summary_annot, dir_out, fnames, labels):
    # pie chart of counts in/not in group
    c = sameGrp_counts.loc[sameGrp_counts.importanceRank == 'top10', 'count'][0]
    df_counts = pd.Series({labels[0]: c,
                           labels[1]: feat_summary_annot.shape[0] - c})
    plotCountsPie(df_counts,
                  None,
                  fnames[0],
                  dir_out)

    # pie chart of feature source
    c = sameGrp_src_counts.loc[sameGrp_src_counts.importanceRank == 'top10',]
    df_counts = pd.Series(c['count'].values, index=c['source'])
    plotCountsPie(df_counts,
                  None,
                  fnames[1],
                  dir_out,
                  colors=[src_colors[s] for s in df_counts.index])

    #heatmap
    s1 = feat_summary_annot.columns.str.startswith('inSame')
    s2 = ~feat_summary_annot.columns.str.contains('top')
    df = feat_summary_annot.loc[:, s1 & s2]

    plt.figure()
    fig, ax = plt.subplots(figsize=(6, 5))
    ax = sns.heatmap(df, yticklabels=False, xticklabels=list(range(1, 11)), vmin=-1, vmax=1, cmap='RdBu', cbar=False)
    ax.set(xlabel='$\it{n}$th Feature', ylabel='Target genes')
    plt.tight_layout()
    plt.savefig("%s/%s_heatmap.png" % (dir_out, fnames[1]))
    plt.close()

In [28]:
# Paralog feature analyze
anlyz_varExp_feats_m(df_featSummary,gs_dir = '../out', outdir_sub='../out')

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [29]:
# on same gene
feat_summary_annot_para = pd.read_csv('../out/areparalog/feat_summary_annot.csv', header=0, index_col=0)
sameGrp_counts, sameGrp_src_counts = getGrpCounts_fromFeatSummaryAnnot(feat_summary_annot_para)
gen_feat_pies(sameGrp_counts,sameGrp_src_counts,feat_summary_annot_para,
              dir_out, ['fig2-para',  'fig2-para'], ['Are paralog','Not paralog'])

<Figure size 432x288 with 0 Axes>